# Snowflake ML with PyTorch Distributor - クイックスタートガイド

このNotebookでは、PyTorch Distributorを使用したSnowflake MLの包括的な機械学習プラットフォームを活用して、エンドツーエンドのMLワークフローを構築します。

このレコメンデーション・エンジンは、Tasty Bytesのフードトラックが販売する各メニューについて、ロイヤルティ顧客にレコメンデーション・スコアを提供する。この出力は、パーソナライズされたアウトリーチ、顧客が訪問するトラックブランドの増加、業績不振のトラックへのトラフィックの増加に使用される。
レコメンデーション・エンジンのモデル学習は、GPUデバイス間での分散学習を活用しており、エンドツーエンドのモデル開発とデプロイは、以下のSnowflake機能を使用して簡素化および合理化されています：

## 学習内容

- **Snowflake Notebooks with GPU Container Runtime (GA)**: Snowflake Notebooks
- **Snowflake Feature Store (GA)** : 特徴量の一元管理と再利用 
- **Snowflake Modeling API (GA)** : Preprocessing, Training (PyTorch API), Evaluation
- **Snowflake Model Registry (GA)** : モデルのバージョン管理、メタデータ管理、デプロイメント
- **Model Deployment from Registry to SPCS (GA)** : モデルの推論サービス

## 構築するもの

以下を使用してエンドツーエンドの深層学習推薦システムを構築します：
- GPU加速による分散PyTorch学習
- Snowflake Feature Storeを使用した特徴量エンジニアリング
- Snowpark Container Services (SPCS)へのモデルデプロイメント
- 包括的なモデル監視と説明可能性


## 1. 環境セットアップとGPU確認

Container Runtime環境では、必要なパッケージが事前にインストールされています。まずGPUの利用可能性を確認します。


In [ ]:
import torch
import warnings
warnings.filterwarnings('ignore')

# GPU デバイス情報の確認
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"✅ 利用可能なGPUデバイス数: {num_gpus}")
    
    for i in range(num_gpus):
        print(f"   デバイス {i}: {torch.cuda.get_device_name(i)}")
    
    # デフォルトデバイスを設定
    torch.cuda.set_device(0)
    print(f"   デフォルトデバイス: {torch.cuda.current_device()}")
else:
    print("❌ CUDAが利用できません。GPU設定を確認してください。")


## 2. 必要なライブラリのインポート

Container Runtime環境では、ML開発に必要なパッケージが事前にインストールされています。


In [ ]:
# 標準ライブラリ
import os
import time
import json
import uuid
from datetime import datetime

# サードパーティライブラリ
import pandas as pd
import numpy as np
import joblib
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, TensorDataset
import shap
import matplotlib.pyplot as plt
import seaborn as sns

# Snowflakeライブラリ
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T

# Snowflake ML
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.preprocessing import LabelEncoder, MinMaxScaler
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.distributors.pytorch import (
    PyTorchDistributor, 
    PyTorchScalingConfig, 
    WorkerResourceConfig,
    get_context
)
from snowflake.ml.data.sharded_data_connector import ShardedDataConnector
from snowflake.ml.modeling.metrics import (
    roc_auc_score,  
    precision_score, 
    recall_score, 
    confusion_matrix
)

print("✅ すべてのライブラリが正常にインポートされました！")
print(f"PyTorchバージョン: {torch.__version__}")
print(f"CUDA利用可能: {torch.cuda.is_available()}")


## 3. Snowflakeセッションの初期化

アクティブなSnowflakeセッションを取得し、クエリタグを設定します。


In [ ]:
# アクティブセッションの取得
session = get_active_session()

# クエリタグの設定（デバッグとパフォーマンス監視に役立ちます）
session.query_tag = {
    "origin": "pytorch_ml_quickstart",
    "name": "end_to_end_recommendation_system",
    "version": {"major": 1, "minor": 0},
    "attributes": {"is_quickstart": 1, "source": "notebook"}
}

# データベース情報の取得
db = str(session.get_current_database().strip('"'))
solution_prefix = (db.upper()).split('_PROD')[0]

print(f"✅ Snowflakeセッション初期化完了")
print(f"   データベース: {db}")
print(f"   ソリューションプレフィックス: {solution_prefix}")
print(f"   現在のロール: {session.get_current_role()}")
print(f"   現在のウェアハウス: {session.get_current_warehouse()}")


## 4. Feature Storeのセットアップ

Snowflake Feature Storeは、データサイエンスと機械学習ワークロードのための特徴量の作成、保存、管理をより簡単で効率的にするように設計されています。

**主要なメリット:**
- **一元化された特徴量管理**: 特徴量を一箇所で保存・管理
- **特徴量の再利用**: チームやプロジェクト間での特徴量の共有
- **一貫性**: 学習と推論での特徴量の一貫性を保証
- **コラボレーション**: データチーム間のサイロを解消


In [ ]:
# Feature Storeスキーマと権限の設定
setup_sql = f"""
USE ROLE ACCOUNTADMIN;
SET FS_ROLE_PRODUCER = '{solution_prefix}_FS_PRODUCER';
SET FS_ROLE_CONSUMER = '{solution_prefix}_DATA_SCIENTIST';
SET FS_DATABASE = '{solution_prefix}_PROD';
SET FS_SCHEMA = 'FS_SCHEMA';
SET SCHEMA_FQN = CONCAT($FS_DATABASE, '.', $FS_SCHEMA);
SET FS_WAREHOUSE = '{solution_prefix}_DS_WH';

-- スキーマの作成
CREATE SCHEMA IF NOT EXISTS IDENTIFIER($FS_SCHEMA);

-- ロール階層の構築
GRANT ROLE IDENTIFIER($FS_ROLE_CONSUMER) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);

-- PRODUCER ロール権限の付与
GRANT USAGE ON DATABASE IDENTIFIER($FS_DATABASE) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT USAGE ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT CREATE DYNAMIC TABLE ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT CREATE VIEW ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT CREATE TAG ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT CREATE DATASET ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT APPLY TAG ON ACCOUNT TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);

-- CONSUMER ロール権限の付与
GRANT USAGE ON DATABASE IDENTIFIER($FS_DATABASE) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT USAGE ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT SELECT,MONITOR ON FUTURE DYNAMIC TABLES IN SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT SELECT,MONITOR ON ALL DYNAMIC TABLES IN SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT SELECT,REFERENCES ON FUTURE VIEWS IN SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT SELECT,REFERENCES ON ALL VIEWS IN SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT CREATE DATASET ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT USAGE ON WAREHOUSE IDENTIFIER($FS_WAREHOUSE) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
"""

# SQLコマンドの実行
for sql_command in setup_sql.strip().split(';'):
    if sql_command.strip():
        try:
            session.sql(sql_command.strip()).collect()
        except Exception as e:
            print(f"警告: {sql_command[:50]}... の実行中にエラー: {e}")

print("✅ Feature Storeスキーマと権限の設定完了")


In [ ]:
# Feature Store Producer ロールを使用してFeature Storeを初期化
session.sql(f'USE ROLE {solution_prefix}_FS_PRODUCER').collect()
session.sql(f'USE WAREHOUSE {solution_prefix}_DS_WH').collect()
session.sql('USE SCHEMA FS_SCHEMA').collect()

# Feature Storeの初期化
fs = FeatureStore(
    session=session,
    database=f"{solution_prefix}_PROD",
    name="FS_SCHEMA",
    default_warehouse=f"{solution_prefix}_DS_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

print("✅ Feature Store初期化完了")


## 5. 特徴量データの準備

推薦システムに必要な特徴量を準備します：
- 顧客特徴量（人口統計学的情報）
- メニュー特徴量（商品情報）
- 購入特徴量（購入履歴統計）


In [ ]:
# メニュー特徴量の取得
menu_spdf = session.sql("""
    SELECT 
        menu_type, 
        truck_brand_name, 
        menu_item_name, 
        item_category, 
        item_subcategory, 
        sale_price_usd 
    FROM raw_pos.menu 
    WHERE item_category != 'Beverage'
""")

# 顧客特徴量の取得
cust_spdf = session.sql("""
    SELECT 
        customer_id, 
        city, 
        country, 
        gender, 
        marital_status, 
        birthday_date, 
        DATEDIFF(year, birthday_date, CURRENT_DATE()) AS age 
    FROM raw_customer.customer_loyalty
""")

# 購入統計特徴量の計算
avg_monthly_purchase = session.sql(f"""
    SELECT  
        customer_id, 
        ROUND(SUM(order_total) / (TIMESTAMPDIFF(MONTH, MIN(date), MAX(date)) + 1), 2) AS avg_monthly_purchase_amount 
    FROM {solution_prefix}_PROD.ANALYTICS.ORDERS_V 
    GROUP BY customer_id
""")

avg_weekly_purchase = session.sql(f"""
    SELECT 
        customer_id,
        ROUND(SUM(order_total) / (TIMESTAMPDIFF(WEEK, MIN(date), MAX(date)) + 1), 2) AS avg_weekly_purchase_amount 
    FROM {solution_prefix}_PROD.ANALYTICS.ORDERS_V 
    GROUP BY customer_id
""")

avg_yearly_purchase = session.sql(f"""
    SELECT 
        customer_id, 
        ROUND(SUM(order_total)/(TIMESTAMPDIFF(YEAR, MIN(date), MAX(date)) + 1), 2) AS avg_yearly_purchase_amount 
    FROM {solution_prefix}_PROD.ANALYTICS.ORDERS_V 
    GROUP BY customer_id
""")

# 購入統計の結合
cust_avgs_spdf = avg_monthly_purchase.join(avg_weekly_purchase, "CUSTOMER_ID").join(avg_yearly_purchase, "CUSTOMER_ID")

print(f"✅ 特徴量データ準備完了")
print(f"   メニュー特徴量: {menu_spdf.count()} レコード")
print(f"   顧客特徴量: {cust_spdf.count()} レコード")
print(f"   購入統計: {cust_avgs_spdf.count()} レコード")


## 6. Feature Store エンティティ・特徴量ビュー・データセット構築

Feature Storeでエンティティと特徴量ビューを作成し、学習用データセットを構築します。


In [ ]:
# エンティティの作成と登録
customer_entity = Entity(name="CustomerIds", join_keys=["Customer_ID"])
fs.register_entity(customer_entity)

menu_entity = Entity(name="Menu_ItemNames", join_keys=["MENU_ITEM_NAME"])
fs.register_entity(menu_entity)

purchase_entity = Entity(name="Purchase_Avgs", join_keys=["Customer_ID"])
fs.register_entity(purchase_entity)

# 特徴量ビューの作成と登録
customer_fv = FeatureView(
    name="CUSTOMER_FEATURES",
    entities=[customer_entity],
    feature_df=cust_spdf,
    refresh_freq="1 day"
)
fs.register_feature_view(feature_view=customer_fv, version="V1", block=True, overwrite=True)

menu_fv = FeatureView(
    name="MENU_FEATURES",
    entities=[menu_entity],
    feature_df=menu_spdf,
    refresh_freq="1 day"
)
fs.register_feature_view(feature_view=menu_fv, version="V1", block=True, overwrite=True)

purchase_fv = FeatureView(
    name="PURCHASE_FEATURES",
    entities=[purchase_entity],
    feature_df=cust_avgs_spdf,
    refresh_freq="1 day"
)
fs.register_feature_view(feature_view=purchase_fv, version="V1", block=True, overwrite=True)

# データサイエンティストロールに切り替え
session.sql(f'USE ROLE {solution_prefix}_DATA_SCIENTIST').collect()

# Feature Storeの再初期化
fs = FeatureStore(
    session=session,
    database=f"{solution_prefix}_PROD",
    name="FS_SCHEMA",
    default_warehouse=f"{solution_prefix}_DS_WH"
)

# 特徴量ビューの取得
customer_fv = fs.get_feature_view(name='CUSTOMER_FEATURES', version='V1')
menu_fv = fs.get_feature_view(name='MENU_FEATURES', version='V1')
purchase_fv = fs.get_feature_view(name='PURCHASE_FEATURES', version='V1')

print("✅ Feature Store エンティティ・特徴量ビュー作成完了")


### 6.1. 特徴量エンジニアリング
このモデルは、カテゴリ（スパース）特徴の埋め込みを作成する。カテゴリ値は一意な整数としてエンコードされる。前処理として、疎な特徴にはラベル符号化を、数値（密な）特徴には最小-最大スケーリングを適用します。

**Snowflakeの特徴**: Snowpark ML Modeling API - 特徴エンジニアリングと前処理（GA） - 頻繁に使用されるscikit-learnの前処理関数を分散実行することで、パフォーマンスとスケーラビリティを向上。

In [ ]:
# 特徴量の定義
sparse_features = [
    'MENU_ITEM_NAME', 'MENU_TYPE', 'TRUCK_BRAND_NAME', 
    'ITEM_CATEGORY', 'ITEM_SUBCATEGORY', 'CITY', 'COUNTRY', 'GENDER', 'MARITAL_STATUS'
]
dense_features = [
    'SALE_PRICE_USD', 'AGE', 'AVG_MONTHLY_PURCHASE_AMOUNT', 
    'AVG_WEEKLY_PURCHASE_AMOUNT', 'AVG_YEARLY_PURCHASE_AMOUNT'
]
label_col = "PURCHASED"

# データセット作成関数
def create_dataset(spine_df, name):
    data = fs.generate_dataset(
        name=name,
        spine_df=spine_df,
        features=[customer_fv, menu_fv, purchase_fv]
    )
    return data.read.to_snowpark_dataframe().drop("BIRTHDAY_DATE")

# 相互作用データセットの分割
interaction_df = session.table('analytics.loyalty_purchased_items')
datasets = interaction_df.random_split([0.1, 0.1, 0.8])
train_df = create_dataset(datasets[0], "pytr_train")
val_df = create_dataset(datasets[1], "pytr_validation")

# 前処理パイプラインの作成
from snowflake.ml.modeling.preprocessing import LabelEncoder, MinMaxScaler
from snowflake.ml.modeling.pipeline import Pipeline

pipeline_steps = []
for i, feat in enumerate(sparse_features):
    pipeline_steps.append((f"LE_{i+1}", LabelEncoder(input_cols=[feat], output_cols=[feat])))

pipeline_steps.append(("SCALER", MinMaxScaler(
    feature_range=(0, 1), input_cols=dense_features, output_cols=dense_features
)))

preprocessing_pipeline = Pipeline(steps=pipeline_steps)
train_data = preprocessing_pipeline.fit(train_df).transform(train_df)
val_data = preprocessing_pipeline.transform(val_df)

# 前処理済みデータを一時テーブルに保存
train_data.write.mode("overwrite").save_as_table("ml.pytr_train_data")
val_data.write.mode("overwrite").save_as_table("ml.pytr_val_data")

print("✅ データセット構築・前処理完了")


### パイプラインの保管
保存されたパイプラインは推論における特徴量変換に使用されます。

In [ ]:
# パイプラインを、一元的にアクセス可能な段階に保存する
pipeline_local_path = f'/tmp/dlrm_preprocessor_v1.joblib'
joblib.dump(preprocessing_pipeline, open(pipeline_local_path, 'wb'))
session.file.put(pipeline_local_path, 
                 '@ML.ML_STAGE/dlrm_preprocessor_v1.joblib', 
                 auto_compress=False, 
                 overwrite=True)

In [ ]:
USE SCHEMA ML;
CREATE or replace STAGE UDF_STAGE;

In [ ]:
import json

data = train_df[dense_features + sparse_features + [label_col]]
data = data.with_columns(sparse_features,
                        [F.col(c).cast(T.StringType()) for c in sparse_features])

def serialize_label_encoders(label_encoders):
    serialized_label_encoders = {}
    for feat, lbe in label_encoders.items():
        serialized_label_encoders[feat] = {
            'input_cols': lbe.input_cols,
            'output_cols': lbe.output_cols,
            'classes_': lbe.classes_.tolist()
        }
    return serialized_label_encoders

def save_label_encoders_to_stage(label_encoders, stage_name, dir_name):
        serialized_label_encoders = json.dumps(label_encoders)
        # Write serialized encoders to a local file first
        with open('/tmp/label_encoders.json', 'w') as f:
            f.write(serialized_label_encoders)
        # Upload the local file to the Snowflake stage
        session.file.put('/tmp/label_encoders.json', f'@{stage_name}/{dir_name}',auto_compress=False)
        return f'Uploaded to @{stage_name}/{dir_name}'
    
label_encoders = {}

# Iterate over each sparse feature
for feat in sparse_features:
    # Initialize LabelEncoder for the current feature
    lbe = LabelEncoder(input_cols=[feat], output_cols=[feat+'_ENCODED'],drop_input_cols=True)
    
    # Fit LabelEncoder to the data
    lbe.fit(data)
    
    # Store the LabelEncoder object for reference
    label_encoders[feat] = lbe
    data = lbe.transform(data)
# Serialize label encoders
serialized_label_encoders = serialize_label_encoders(label_encoders)
stage_name="UDF_STAGE"
dir_name="dlrm_label_encoders"
# Save serialized label encoders to a file
with open('/tmp/label_encoders.json', 'w') as f:
    json.dump(serialized_label_encoders, f)
save_label_encoders_to_stage(serialized_label_encoders, stage_name, dir_name)

## 7. モデル定義・学習

このPyTorchモデルは深層学習推薦モデル（DLRM）である。これは、各ロイヤリティ顧客に全てのメニューの推薦スコアを提供するために使用されている。

- 埋め込み層はカテゴリ特徴を密なベクトルに変換する。
- 数値的特徴は多層パーセプトロン（MLP）層を通して処理される。
- 特徴相互作用層は入力特徴のペア間の複雑な関係を捉える。
- 最後のdence層は推薦スコアを生成する。


In [ ]:
# 深層学習推薦モデル（DLRM）の定義
class FeatureInteraction(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, inputs):
        feature_dim = inputs.shape[1]
        concat_features = inputs.view(-1, feature_dim, 1)
        dot_products = torch.matmul(concat_features, concat_features.transpose(1, 2))
        ones = torch.ones_like(dot_products) 
        mask = torch.triu(ones)
        out_dim = feature_dim * (feature_dim + 1) // 2
        flat_result = dot_products[mask.bool()]
        reshape_result = flat_result.view(-1, out_dim)
        return reshape_result

class DLRM(nn.Module):
    
    def __init__(self, sparse_feature_number, dense_feature_number, num_embeddings, embed_dim, bottom_mlp_dims, top_mlp_dims):
        super(DLRM, self).__init__()
        
        self.embeddings = nn.EmbeddingBag(num_embeddings=num_embeddings, embedding_dim=embed_dim, mode='sum')
        self.layer_feature_interaction = FeatureInteraction()
        
        self.bottom_mlp = torch.nn.Sequential(
            torch.nn.Linear(dense_feature_number, bottom_mlp_dims[0]),
            torch.nn.ReLU(),
            torch.nn.Linear(bottom_mlp_dims[0], bottom_mlp_dims[1]),
            torch.nn.ReLU()
        )
        
        top_mlp_input_dim = (
            (embed_dim + bottom_mlp_dims[1]) 
            * ((embed_dim + bottom_mlp_dims[1]) + 1) // 2 
            + bottom_mlp_dims[1]
         )

        self.top_mlp = nn.Sequential(
            nn.Linear(top_mlp_input_dim, top_mlp_dims[0]),
            nn.ReLU(),
            nn.Linear(top_mlp_dims[0], top_mlp_dims[1]),
            nn.ReLU(),
            nn.Linear(top_mlp_dims[1], 1)
        )

    def forward(self, x_sparse, x_dense):
        # Embedding layer for categorical inputs
        embed_x = self.embeddings(x_sparse)
        # MLPs for numeric inputs
        bottom_mlp_output = self.bottom_mlp(x_dense)
        # Combine categical embeddings and MLP outputs
        concat_first = torch.cat([bottom_mlp_output, embed_x], dim=-1)
        # Get feature interactions
        interaction = self.layer_feature_interaction(concat_first)
        # Concat interaction outputs with MLP outputs
        concat_second = torch.cat([interaction, bottom_mlp_output], dim=-1)
        # MLP layers to output 
        output = self.top_mlp(concat_second)
        return output

print("✅ DLRMモデル定義完了")


### 7.1 モデルトレーニング

モデルトレーニング機能は、各デバイスにモデルとデータを配置する。勾配は各トレーニングバッチで結合され、すべてのデバイスに伝搬される。各エポックの後、トレーニングと検証の損失がすべてのデバイスで平均化され、モデルの重みが保存されます。

**Snowflakeの特長**: Snowpark ML Modeling API - PyTorch - Snowpark DataFrameからGPUデバイスに分散して実行。

In [ ]:
# トレーニングデータのエポック数とレコード数を調整
num_epochs = 2
training_sample = 100000

In [ ]:
# 初期化
def setup(rank, world_size):
    #処理グループの初期化
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

    torch.manual_seed(42)

# 分散学習関数とPyTorch Distributor実行
def distributed_training_function():
    """Container Runtime環境に最適化された分散学習関数"""
    context = get_context()
    rank = context.get_rank()
    world_size = context.get_world_size()
    # local_rank = context.get_local_rank()    
    
    # 分散学習の初期化
    setup(rank, world_size)
    # dist.init_process_group("nccl", rank=rank, world_size=world_size)
    # torch.cuda.set_device(local_rank)
    # torch.manual_seed(42 + rank)
    
    batch_size = 256
    
    # データの取得
    dataset_map = context.get_dataset_map()
    training_data = dataset_map["train"].get_shard().to_torch_datapipe(batch_size=batch_size, shuffle=True)
    validation_data = dataset_map["val"].get_shard().to_torch_datapipe(batch_size=batch_size, shuffle=False)
    
    train_loader = DataLoader(training_data, batch_size=None)
    val_loader = DataLoader(validation_data, batch_size=None)
    
    # モデルの初期化
    model = DLRM(
        sparse_feature_number=len(sparse_features),
        dense_feature_number=len(dense_features),
        num_embeddings=142,
        embed_dim=128,
        bottom_mlp_dims=[256, 128],
        top_mlp_dims=[128,128]
    )
    
    model = model.to(rank)
    ddp_model = DDP(model, device_ids=[rank])
    
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(ddp_model.parameters(), lr=0.001, weight_decay=0.01)
    
    best_val_loss = float('inf')
    
    # 学習ループ
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        ddp_model.train()
        total_train_loss = 0.0
        num_train_batches = 0
        
        for batch_idx, batch_data in enumerate(train_loader):
            y = batch_data.pop(label_col).type(torch.float32).to(rank).squeeze()
            
            x_sparse = torch.stack(
                [tensor.to(torch.int).squeeze() for key, tensor in batch_data.items() if key in sparse_features],
                dim=1
            )
            x_dense = torch.stack(
                [tensor.to(torch.float32).squeeze() for key, tensor in batch_data.items() if key in dense_features],
                dim=1
            )
            
            optimizer.zero_grad()
            outputs = ddp_model(x_sparse, x_dense)
            loss = criterion(outputs, y.unsqueeze(1))
            loss.backward()
            # torch.nn.utils.clip_grad_norm_(ddp_model.parameters(), max_norm=1.0)
            optimizer.step()
            
            total_train_loss += loss.item()
            num_train_batches += len(y)
            
            if batch_idx % 100 == 0 and rank == 0:
                print(f"📈 エポック {epoch+1}/{num_epochs}, バッチ {batch_idx}, 損失: {loss.item():.4f}")
        
        # 平均学習損失の計算
        avg_train_loss = total_train_loss / num_train_batches
        train_loss_tensor = torch.tensor(avg_train_loss, device=rank)
        dist.all_reduce(train_loss_tensor)
        dist.barrier()
        avg_train_loss = train_loss_tensor.item() / world_size
        
        # 検証フェーズ
        ddp_model.eval()
        total_val_loss = 0.0
        num_val_batches = 0
        
        with torch.no_grad():
            for batch_data in val_loader:
                y_val = batch_data.pop(label_col).float().to(rank).squeeze()
                x_sparse_val = torch.stack([
                    batch_data[feat].long().squeeze() for feat in sparse_features
                ], dim=1).to(rank)
                x_dense_val = torch.stack([
                    batch_data[feat].float().squeeze() for feat in dense_features
                ], dim=1).to(rank)
                
                output_val = ddp_model(x_sparse_val, x_dense_val)
                loss_val = criterion(output_val, y_val.unsqueeze(1))
                total_val_loss += loss_val.item()
                num_val_batches += 1
        
        # 平均検証損失の計算
        avg_val_loss = total_val_loss / num_val_batches if num_val_batches > 0 else 0.0
        val_loss_tensor = torch.tensor(avg_val_loss, device=rank)
        dist.all_reduce(val_loss_tensor)
        dist.barrier()
        avg_val_loss = val_loss_tensor.item() / world_size
        
        if rank == 0:
            epoch_time = time.time() - epoch_start_time
            print(f"🎯 エポック {epoch+1}/{num_epochs} 完了:")
            print(f"   学習損失: {avg_train_loss:.4f}")
            print(f"   検証損失: {avg_val_loss:.4f}")
            print(f"   時間: {epoch_time:.2f}秒")
            
            # 最良モデルの保存
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_loss': avg_train_loss,
                    'val_loss': avg_val_loss
                }, '/tmp/pytr_best_model.pth')
                print(f"💾 新しい最良モデル保存 (検証損失: {best_val_loss:.4f})")
    
    dist.destroy_process_group()
    
    if rank == 0:
        print("🎉 学習完了！")
        print(f"🏆 最良検証損失: {best_val_loss:.4f}")

# PyTorch Distributorの設定と実行
pytorch_distributor = PyTorchDistributor(
    train_func=distributed_training_function,
    scaling_config=PyTorchScalingConfig(
        num_nodes=1,
        num_workers_per_node=1,
        resource_requirements_per_worker=WorkerResourceConfig(num_cpus=4, num_gpus=1)
    )
)

# 学習データの準備
train_table = session.table("ml.pytr_train_data")
val_table = session.table("ml.pytr_val_data")
train_connector = ShardedDataConnector.from_dataframe(train_table.limit(training_sample))
val_connector = ShardedDataConnector.from_dataframe(val_table)

print("🏋️ 分散学習開始...")

# 分散学習の実行
training_result = pytorch_distributor.run(
    dataset_map={"train": train_connector, "val": val_connector}
)

print("✅ 分散学習完了！")


## 8. モデルの登録とデプロイ

モデルは Snowflake Model Registry に記録されます。ログに記録されたモデルは、Snowpark Container Services (SPCS)上で推論のためにデプロイされます。

**Snowflakeの特長**： Snowflake Model Registry with SPCS deployment - 柔軟なコンピュート環境において、Snowflakeでモデルとそのメタデータを安全にデプロイ、管理します。

![model_serving](https://docs.snowflake.com/en/_images/model-registry-spcs-deployment.png)


In [ ]:
# 学習済みモデルの読み込みとModel Registry登録
def load_trained_model(model_path):
    model = DLRM(
        sparse_feature_number=len(sparse_features),
        dense_feature_number=len(dense_features),
        num_embeddings=142, embed_dim=128,
        bottom_mlp_dims=[256, 128], top_mlp_dims=[128, 128]
    )
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    return model, checkpoint

# 学習済みモデルの読み込み
model, checkpoint = load_trained_model('/tmp/pytr_best_model.pth')

# Model Registryの初期化
registry = Registry(
    session=session, 
    database_name=f"{solution_prefix}_PROD", 
    schema_name="REGISTRY"
)

# サンプル入力データの準備
train_table = session.table("ml.pytr_train_data")
sample_input = train_table.limit(1).to_pandas()
x_sparse = torch.tensor(sample_input[sparse_features].values, dtype=torch.int)
x_dense = torch.tensor(sample_input[dense_features].values, dtype=torch.float32)

# モデルをレジストリに登録
model_ref = registry.log_model(
    model,
    model_name="PyTorchRecModel",
    version_name="V1",
    pip_requirements=["torch==2.6.0", "torchvision==0.21.0"],
    sample_input_data=[x_sparse[0].unsqueeze(0), x_dense[0].unsqueeze(0)],
    options={'relax_version': False, 'multiple_inputs': True}
)

# モデルメタデータの追加
model_ref.description = "PyTorch Deep Learning Recommendation System - Container Runtime最適化"

# 学習メトリクスの追加
model_ref.set_metric("train_loss", round(checkpoint['train_loss'], 4))
model_ref.set_metric("val_loss", round(checkpoint['val_loss'], 4))
model_ref.set_metric("epochs_trained", checkpoint['epoch'] + 1)

print(f"✅ モデル登録完了: {model_ref.model_name} v{model_ref.version_name}")
print(f"   最終学習損失: {checkpoint['train_loss']:.4f}")
print(f"   最終検証損失: {checkpoint['val_loss']:.4f}")
print(f"   学習エポック: {checkpoint['epoch'] + 1}")


In [ ]:
model_ref.create_service(
    service_name="PYTORCH_RECOMMENDATION_SERVICE",
    service_compute_pool=f"{solution_prefix}_DEPLOY_POOL",
    image_repo=f"{solution_prefix}_PROD.REGISTRY.IMAGE_REPO",
    build_external_access_integration=f"{solution_prefix}_CONDA_ACCESS_INTEGRATION"
)
print("✅ モデルがSPCSに正常にデプロイされました！")


## 9. モデルの推論と検証

テストデータの推論は、専用のコンピュートプール上で動作する SPCS にデプロイされたモデルを使用して完了する。テストデータのフィーチャーはフィーチャーストアからアクセスされ、前処理パイプラインが必要に応じてデータを変換します。

**モデル出力**： モデルは入力特徴量に基づくスコアを出力する。スコアが高ければ高いほど、その顧客にとってそのメニューがよりおすすめであることを示します。モデルのパフォーマンスを評価するために、スコアからバイナリ予測が作成されます（スコアが0.5以上の場合は1、そうでない場合は0）。

**スノーフレークの特徴：**

- SPCS 上での推論（PuPr） - 専用のコンピュートプールを持つコンテナ環境にデプロイされたモデルに対して推論を実行します。
- Snowpark ML Modeling API - Evaluation Metrics (GA) - 使用頻度の高い scikit-learn 前処理関数の分散実行により、パフォーマンスとスケーラビリティを向上。

In [ ]:
# テストデータでの推論と評価
test_df = create_dataset(datasets[2], "pytr_test")
test_df_subset = test_df.sample(n=5000)
test_data = preprocessing_pipeline.transform(test_df_subset)
test_data_pd = test_data.to_pandas()

# 推論用入力データの準備
sparse_input = torch.tensor(test_data_pd[sparse_features].values, dtype=torch.long)
dense_input = torch.tensor(test_data_pd[dense_features].values, dtype=torch.float32)

# SPCS推論
predictions = model_ref.run(
    [sparse_input, dense_input], 
    function_name="forward", 
    service_name="PYTORCH_RECOMMENDATION_SERVICE"
)
predictions['output_feature_0'] = predictions['output_feature_0'].apply(
    lambda x: x[0] if isinstance(x, list) else float(x)
)
st.success("✅ SPCS推論完了")
    
# 評価データフレームの作成
eval_df_pd = pd.concat([
    test_data_pd[["CUSTOMER_ID", "MENU_ITEM_NAME", "PURCHASED"]], 
    predictions.rename(columns={'output_feature_0': 'PREDICTION'})
], axis=1).assign(
    BINARY_PREDICTION=lambda df: np.where(df['PREDICTION'] >= 0.5, 1, 0)
)

eval_df = session.create_dataframe(eval_df_pd)

# 評価メトリクスの計算
auc = roc_auc_score(df=eval_df, y_true_col_names="PURCHASED", y_score_col_names="PREDICTION")
precision = precision_score(df=eval_df, y_true_col_names="PURCHASED", y_pred_col_names="BINARY_PREDICTION")
recall = recall_score(df=eval_df, y_true_col_names="PURCHASED", y_pred_col_names="BINARY_PREDICTION")

# Streamlitでメトリクス表示
st.subheader("📊 モデルパフォーマンス")
cols = st.columns(3)
cols[0].metric("AUC", round(auc, 3))
cols[1].metric("Recall", round(recall, 3))
cols[2].metric("Precision", round(precision, 3))

# モデルレジストリにメトリクスを更新
model_ref.set_metric("AUC", round(auc, 4))
model_ref.set_metric("Precision", round(precision, 4))
model_ref.set_metric("Recall", round(recall, 4))

f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
model_ref.set_metric("F1_Score", round(f1_score, 4))


In [ ]:
# 🎉 Snowflake ML クイックスタート完了サマリー

print("🎉 Snowflake ML クイックスタート完了！")
print("="*60)
print(f"📊 最終モデルパフォーマンス:")
print(f"   AUC: {auc:.4f}")
print(f"   Precision: {precision:.4f}")
print(f"   Recall: {recall:.4f}")
print(f"   F1 Score: {f1_score:.4f}")
print(f"")
print(f"🏆 モデル情報:")
print(f"   名前: {model_ref.model_name}")
print(f"   バージョン: {model_ref.version_name}")
print(f"   学習エポック: {checkpoint['epoch'] + 1}")
print(f"   最終検証損失: {checkpoint['val_loss']:.4f}")
print(f"")
print(f"🔧 使用した技術:")
print(f"   - Snowflake Notebooks on Container Runtime")
print(f"   - PyTorch Distributor (分散学習)")
print(f"   - Snowflake Feature Store")
print(f"   - Snowflake Model Registry")
print(f"   - SPCS デプロイメント")
print("="*60)
print("✅ すべてのコンポーネントが正常に動作しました！")
print("")
print("🚀 次のステップ:")
print("   1. ハイパーパラメータチューニングでパフォーマンス向上")
print("   2. A/Bテストによる本番環境でのモデル比較")
print("   3. リアルタイム監視でデータドリフト検出")
print("   4. 自動再学習パイプラインの構築")
print("")
print("📖 Container Runtime for ML の詳細:")
print("   https://docs.snowflake.com/en/developer-guide/snowflake-ml/notebooks-on-spcs")

# セッションのクローズ（Notebook環境では通常クローズしない）
# session.close()


### Clean up

In [ ]:
# # SPCS Model Service
# model_ref.delete_service(
#     service_name="PYTORCH_RECOMMENDATION_SERVICE"
# )

# # Model
# registry.delete_model(
#     model_name="PyTorchRecModel"
# )

# # Feature View
# fs.delete_feature_view(customer_fv)
# fs.delete_feature_view(menu_fv)
# fs.delete_feature_view(purchase_fv)


# # Feature Entiry
# fs.delete_entity("CustomerIds")
# fs.delete_entity("Menu_ItemNames")
# fs.delete_entity("Purchase_Avgs")